In [ ]:
# download nltk stopwords
import nltk
nltk.download('stopwords')

# Install a particular version of `google-cloud-storage` because (oddly enough) 
# the  version on Colab and GCP is old. A dependency error below is okay.
!pip install -q google-cloud-storage==1.43.0

# authenticate below for Google Storage access as needed
from google.colab import auth
auth.authenticate_user()
from google.colab import drive
drive.mount('/content/drive/')
import sys
sys.path.append('/content/drive/MyDrive/Information Retrieval/Project/information-retrieval-final-project/colab_cache')
%cd /content/drive/MyDrive/Information Retrieval/Project/information-retrieval-final-project/colab_cache

# install ngrok to emulate public IP / address
!wget -N https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip

# TODO: sign up for an ngrok account
# then put your ngrok token below, uncomment, and execute
!./ngrok authtoken 2KH9iAwVvV0mhmqyNWMhohNLa1x_7CqfV2zBDBTJRGUeFhC7y # 2K5pCuAwvo0Kajtz8DA5HwCRr7f_2Rssap33tUn3qcHuxFDBY

# install a ngrok python package and a version of flask that works with it in 
# colab
!pip -q install flask-ngrok
!pip -q install flask==0.12.2
# !pip -q install flask_restful

# Run the app

In [ ]:
!pip install pyspark

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import numpy as np
from collections import defaultdict, Counter
import pandas as pd
from inverted_index_colab import *
import colab_search_frontend as se
import time
import requests
import importlib
     

In [ ]:
import importlib
import colab_search_frontend as se
importlib.reload(se)
from flask_ngrok import run_with_ngrok
run_with_ngrok(se.app) 
se.app.run()

# Testing your app

Once your app is running you can query it. You can simply do that by navigating to the URL that ngrok gave you above or through code in a different python session. For example, once the frontend app is running, you can navigate to:
http://YOUR_SERVER_DOMAIN/search?query=hello+world where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io, which is printed above in Colab or that is your external IP on GCP.

The code below shows how to issue a query from python. This is also how our testing code will issue queries to your search engine, so make sure to test your search engine this way after you deploy it to GCP and before submission. Command line instructions for deploying your search engine to GCP are available at `run_frontend_in_gcp.sh`. Note that we will not only issue training queries to your search engine, but also test queries, i.e. queries that you've never seen before.

We figgured our model would behave diffrently for long and short queries so we decided to check.

In [ ]:
import json

with open('queries_train.json', 'rt') as f:
  queries = json.load(f)

In [ ]:
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "first", "see", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became", "page", "article",
                    "reference", "source", "content",
                    "fact", "year", "date", "place", "wiki",
                    "edit", "version", "user", "talk", "discussion", "template",
                    "category", "project", "author", "writer",
                    "creator", "publisher", "editor", "publication", "edition",
                    "issue", "chapter"]
all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

def tokenize(text):
    """
    This function aims in tokenize a text into a list of tokens. Moreover, it filter stopwords.

    Parameters:
    -----------
    text: string , represting the text to tokenize.

    Returns:
    -----------
    list of tokens (e.g., list of tokens).
    """
    list_of_tokens = [token.group() for token in RE_WORD.finditer(text.lower()) if token.group() not in all_stopwords]
    return list_of_tokens

In [ ]:
title_queries = [(q, true_wids) for q, true_wids in queries.items()]



here we divide the queries given to us into two groups, more than 3 words and 2 or less. note that we first tokenized the queries.

In [ ]:
# divide into long and short queries
short_queries = [title_queries[i] for i in range(len(title_queries)) if not ((len(tokenize(title_queries[i][0])) >= 3) or (title_queries[i][0][-1] == '?' and len(title_queries[i][0].split()) >= 4) or (len(title_queries[i][0].split()) >= 5))]
long_queries = [title_queries[i] for i in range(len(title_queries)) if (len(tokenize(title_queries[i][0])) >= 3) or (title_queries[i][0][-1] == '?' and len(title_queries[i][0].split()) >= 4) or (len(title_queries[i][0].split()) >= 5)]


with such small data its hard to get a good split into train and test so we decided to leave one out for test and train on the rest.


In [ ]:
# divide into train and test (test is just one query)
train_short = short_queries[:18]
test_short = short_queries[18:]
train_long = long_queries[:12]
test_long = long_queries[12:]
train_long
for word, _ in queries.items():
  print(word)

In [ ]:
def average_precision(true_list, predicted_list, k=40):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    precisions = []

    for i,doc_id in enumerate(predicted_list):        
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)            
            precisions.append(prec)
    if len(precisions) == 0:
        return 0.0
    return round(sum(precisions)/len(precisions),3)

In [ ]:
from traitlets.traitlets import Long
import requests
from time import time
# http://104.197.198.70:8080/search_body?query=american
# url = 'http://35.232.59.3:8080'
# place the domain you got from ngrok or GCP IP below. 
url = 'http://104.197.198.70:8080.ngrok.io'

qs_res = []
for q, true_wids in queries.items():
  duration, ap = None, None
  t_start = time()
  try:
    #res = requests.get(url + '/search', {'query': q}, timeout=35)
    res = requests.get('http://104.197.198.70:8080/search?query=' + str(q))

    duration = time() - t_start
    if res.status_code == 200:
      pred_wids, query = zip(*res.json())
      ap = average_precision(true_wids, pred_wids)
  except:
    pass
  
  qs_res.append((q, duration, ap))

We implemented a function that calculates the average of precisions we got per query

In [ ]:
def calc_avg_total(qs_res):
  sum = 0
  nones = 0
  for i in range(len(qs_res)):
    if qs_res[i][-1] != None:
      sum += qs_res[i][-1]
    else:
      nones += 1 
  return sum/(len(qs_res)-nones)

Now all that is left to do is look for the best weights

### Showing the Results:

In [ ]:
data = [[1, 0, 0, 0, 0 ,0.604],
[0 , 1, 0, 0, 0 , 0.328562],
[0 , 0, 1, 0, 0 , 0.40425],
[0 , 0, 0, 1, 0 , 0.410625],
[0 , 0, 0, 0, 1 , 0.350625],
[0.4 , 0, 0.3, 0.3, 0 , 0.46531],
[0.4 , 0.3, 0.3, 0, 0 , 0.63275],
[0.8 , 0.1, 0.1, 0, 0 , 0.63275],
[0.2, 0.2, 0.2, 0.2, 0.2 , 0.46418],
[0.8 , 0.05, 0.05, 0.1, 0 , 0.4949375],
[0.2, 0.4, 0.4, 0, 0 , 0.6366875],
[0.7, 0 ,0.3 ,0 ,0 , 0.620937],
[0.7, 0.3, 0, 0, 0 , 0.61675]]


data.sort(key=lambda x: x[-1])
x_coords = range(1, len(data) + 1)
y_values = [row[-1] for row in data]
x_labels = ["tw:{:.2f}<br>prw:{:.2f}<br>pvw:{:.2f}<br>bw:{:.2f}<br>aw:{:.2f}".format(row[0], row[1], row[2], row[3], row[4]) for row in data]

import plotly.graph_objs as go

colorscale = [[0, 'red'], [1, 'blue']]
trace = go.Bar(x=x_labels, y=y_values, marker=dict(color=y_values, colorscale=colorscale, cmin=min(y_values), cmax=max(y_values), showscale=True, colorbar=dict(title='Score',titlefont=dict(size=18),tickfont=dict(size=18))))
layout = go.Layout(
    title='Short Queries Results',
    titlefont=dict(size=26),
    xaxis=dict(title='Weights',tickmode='array',titlefont=dict(size=22),tickfont=dict(size=18)),
    yaxis=dict(title='M.A.P@40 Score',titlefont=dict(size=22),tickfont=dict(size=18)),
    yaxis_range=[0.3, 0.7]
)
fig = go.Figure(data=[trace], layout=layout)

fig.show()

In [ ]:
data = [[1, 0, 0, 0, 0 , 0.196],
[0, 1, 0, 0, 0 , 0.312769],
[0, 0, 1, 0, 0 , 0.20476],
[0, 0, 0, 1, 0 , 0.27676],
[0, 0, 0, 0, 1 , 0.15275],
[0, 0.5, 0, 0.5, 0 , 0.27992],
[0 , 0.2, 0.2, 0.6, 0 , 0.2771],
[0.2 , 0.1, 0.1, 0.6, 0 , 0.241],
[0, 0.333, 0.333, 0.333, 0 , 0.2805],
[0 , 0.1, 0.1, 0.8, 0 , 0.277],
[0.2, 0.5, 0.25, 0.25, 0 , 0.28130769],
[0, 0.4 ,0.2 ,0.4 ,0 , 0.28007],
[0.1, 0.4, 0.1, 0.4, 0 , 0.2803]]


data.sort(key=lambda x: x[-1])
x_coords = range(1, len(data) + 1)
y_values = [row[-1] for row in data]
x_labels = ["tw:{:.2f}<br>prw:{:.2f}<br>pvw:{:.2f}<br>bw:{:.2f}<br>aw:{:.2f}".format(row[0], row[1], row[2], row[3], row[4]) for row in data]

import plotly.graph_objs as go

colorscale = [[0, 'red'], [1, 'blue']]
trace = go.Bar(x=x_labels, y=y_values, marker=dict(color=y_values, colorscale=colorscale, cmin=min(y_values), cmax=max(y_values), showscale=True, colorbar=dict(title='Score',titlefont=dict(size=18),tickfont=dict(size=18))))
layout = go.Layout(
    title='Long Queries Results',
    titlefont=dict(size=26),
    xaxis=dict(title='Weights',tickmode='array',titlefont=dict(size=22),tickfont=dict(size=18)),
    yaxis=dict(title='M.A.P@40 Score',titlefont=dict(size=22),tickfont=dict(size=18)),
    yaxis_range=[0.1, 0.35]
)
fig = go.Figure(data=[trace], layout=layout)

fig.show()

### Conclusions

1. Our model is much better for shorter queries in terms of mean average precision.

2. Wiki scores for short queries are calculated with a larger emphasis on the title, assisted by pagerank and pageviews (anchor and body had a negative influence on the M.A.P@40 score). We believe this is a good way to answer short queries as those are usually for very specific documents. For long queries we did the opposite, granting the body a higher weight while giving none to the title because we believe most of the long queries are usually aimed for the actuall meaning of the text and whats in it, not the title per say.

3. The training set is very small and our results might not be enough to get a good idea of what the weights should actually be for good results on the actual test set. 

In [ ]:



data.sort(key=lambda x: x[-1])
x_coords = range(1, len(data) + 1)
y_values = [8,2,1]
x_labels = ["Bins Downloaded<br>from bucket<br>on demand","Bins loaded to<br>Instance's SSD ","Title & Body<br>Intermittent Usage<br>(Anchor set aside)"]

import plotly.graph_objs as go

colorscale = [[0, 'red'], [1, 'blue']]
trace = go.Bar(x=x_labels, y=y_values, marker=dict(color=y_values, colorscale=colorscale, cmin=min(y_values), cmax=max(y_values), showscale=True, colorbar=dict(title='Seconds',titlefont=dict(size=18),tickfont=dict(size=18))))
layout = go.Layout(
    title='Runtime Progress',
    titlefont=dict(size=26),
    xaxis=dict(title='Key Implementation Points',tickmode='array',titlefont=dict(size=22),tickfont=dict(size=18)),
    yaxis=dict(title='Average Per Query (Sec)',titlefont=dict(size=22),tickfont=dict(size=18),dtick = 1),
    yaxis_range=[0, 10]
)
fig = go.Figure(data=[trace], layout=layout)

fig.show()